# CSE-CIC-IDS 2017

In [1]:
model_id = "dnn1"

In [2]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob, time, os

import keras

Using TensorFlow backend.
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Us

In [3]:
NOTEBOOK_PATH = "D:/Delta Stuff/Scifair20/"

In [4]:
f_names = glob.glob(NOTEBOOK_PATH + "IDS2017/MachineLearningCVE/*.csv")
df = pd.concat(tqdm((pd.read_csv(f) for f in f_names), total=len(f_names)))

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:255: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  sort=sort,
 38%|███████████████████████████████▌                                                    | 3/8 [00:03<00:05,  1.05s/it]C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:255: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  sort=sort,
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:13<00:00,  1.71s/it]


In [5]:
df.reset_index(drop=True, inplace=True)

In [6]:
df.columns = list(map(lambda x: x.strip(), df.columns))

In [7]:
# Remove variables with no variance
remove_vars = []
for i in tqdm(range(len(df.columns))):
    col = df.columns[i]
    unique = df[col].unique()
    if len(unique) <= 1:
        remove_vars.append(col)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 18.06it/s]


In [8]:
remove_vars

['Bwd PSH Flags',
 'Bwd URG Flags',
 'Fwd Avg Bytes/Bulk',
 'Fwd Avg Packets/Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes/Bulk',
 'Bwd Avg Packets/Bulk',
 'Bwd Avg Bulk Rate']

In [9]:
df.drop(remove_vars, axis=1, inplace=True)

df.drop(["Flow Bytes/s", "Flow Packets/s"], axis='columns', inplace=True)

In [10]:
df.shape

(2830743, 69)

# Standardize data

In [11]:
from sklearn.preprocessing import PowerTransformer, MinMaxScaler

In [12]:
scaler = PowerTransformer(method='yeo-johnson', standardize=True)
# scaler = MinMaxScaler()

In [13]:
x_df = df.drop("Label", axis=1)
y_df = df.loc[:, "Label"]

In [14]:
x_scaled = scaler.fit_transform(x_df, y_df)

In [15]:
scaled_summary = pd.DataFrame(x_scaled, columns=x_df.columns).describe()
scaled_summary

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,...,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06
mean,5.614575e-17,-5.294087e-16,-1.481637e-15,-7.895848e-16,-6.103742e-16,4.526701e-16,-2.069120e-16,-5.198502e-16,-1.929056e-15,1.910080e-16,...,-6.024222e-18,-1.909197e-15,-2.522141e-16,8.514234e-18,2.377560e-17,1.824135e-16,8.731106e-17,-3.527785e-16,-2.910904e-16,1.681963e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-8.085491e+00,-2.286645e+01,-1.304010e+00,-1.655424e+00,-1.617208e+00,-1.349540e+00,-1.591983e+00,-9.953509e-01,-1.640853e+00,-7.155903e-01,...,-1.200386e+00,-2.752211e+01,-4.925726e-01,-2.797894e-01,-4.923377e-01,-4.926216e-01,-5.006239e-01,-2.968475e-01,-5.006238e-01,-5.006238e-01
25%,-8.738939e-01,-8.876095e-01,-2.338725e-01,-4.381973e-01,-5.559671e-01,-1.349540e+00,-6.855578e-01,-9.953509e-01,-6.405733e-01,-7.155903e-01,...,-1.200386e+00,-9.087109e-01,-4.925726e-01,-2.797894e-01,-4.923377e-01,-4.926216e-01,-5.006239e-01,-2.968475e-01,-5.006238e-01,-5.006238e-01
50%,-5.280598e-01,1.134869e-01,-2.338725e-01,1.225253e-01,1.168017e-01,1.793477e-01,1.106896e-01,-1.864637e-01,3.389061e-01,-7.155903e-01,...,1.163599e-01,-3.832149e-01,-4.925726e-01,-2.797894e-01,-4.923377e-01,-4.926216e-01,-5.006239e-01,-2.968475e-01,-5.006238e-01,-5.006238e-01
75%,5.271538e-01,9.319696e-01,8.888288e-01,7.029430e-01,5.919397e-01,5.791860e-01,4.752672e-01,1.140584e+00,5.905137e-01,1.240967e+00,...,6.579127e-01,8.493489e-01,-4.925726e-01,-2.797894e-01,-4.923377e-01,-4.926216e-01,-5.006239e-01,-2.968475e-01,-5.006238e-01,-5.006238e-01
max,1.713337e+00,1.538709e+00,2.168643e+00,3.041949e+00,5.883933e+00,4.035647e+00,3.233995e+00,2.329224e+00,4.685579e+00,1.704225e+00,...,2.511256e+00,3.492139e+01,2.093608e+00,3.576140e+00,2.098229e+00,2.092428e+00,2.006166e+00,3.369526e+00,2.006018e+00,2.006746e+00


In [16]:
y_df_group = y_df.copy()

for i in tqdm(range(len(y_df_group))):
    attack = y_df_group[i]
    group = attack
    if attack in ("DoS Hulk", "DoS GoldenEye", "DoS slowloris", "DoS Slowhttptest"):
        group = "DoS"
    elif attack.startswith("Web Attack"):
        group = "Web Attack"
    elif attack == "FTP-Patator":
        group = "Brute Force FTP"
    elif attack == "SSH-Patator":
        group = "Brute Force SSH"
    
    y_df_group[i] = group

100%|█████████████████████████████████████████████████████████████████████| 2830743/2830743 [02:20<00:00, 20081.11it/s]


In [17]:
y_df.value_counts()

BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Label, dtype: int64

In [18]:
y_df_group.value_counts()

BENIGN             2273097
DoS                 252661
PortScan            158930
DDoS                128027
Brute Force FTP       7938
Brute Force SSH       5897
Web Attack            2180
Bot                   1966
Infiltration            36
Heartbleed              11
Name: Label, dtype: int64

In [19]:
y_df_enc = pd.get_dummies(y_df_group)

In [29]:
x_scaled.to_csv(NOTEBOOK_PATH + "IDS2017/x_tranformed.csv")

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [30]:
x_scaled.shape

(2830743, 68)

In [33]:
pd.DataFrame(x_scaled, columns=x_df.columns).to_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv", index=False)

In [34]:
y_df_enc.to_csv(NOTEBOOK_PATH + "IDS2017/y_1henc.csv", index=False)

# Split data into train and test

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x_train, x_val, y_train, y_val = train_test_split(x_scaled, y_df_enc, test_size = 0.1, random_state = 42)

In [22]:
y_train.sum(axis=0)

BENIGN             2045803
Bot                   1772
Brute Force FTP       7113
Brute Force SSH       5326
DDoS                115176
DoS                 227435
Heartbleed               9
Infiltration            33
PortScan            143020
Web Attack            1981
dtype: int64

In [23]:
y_val.sum(axis=0)

BENIGN             227294
Bot                   194
Brute Force FTP       825
Brute Force SSH       571
DDoS                12851
DoS                 25226
Heartbleed              2
Infiltration            3
PortScan            15910
Web Attack            199
dtype: int64

# Train model

In [24]:
from keras.models import Model, Sequential
from keras.layers import Dense, BatchNormalization, Dropout, InputLayer
from keras.optimizers import SGD, Adam

In [27]:
def addDenseBlock(model, units, **params):
    model.add(Dense(units=units, activation=params['hidden_activation'], input_dim=x_train.shape[1]))
    if params['batch_normalization']: model.add(BatchNormalization())
    if params['dropout_rate'] > 0: model.add(Dropout(params['dropout_rate']))

def createModel(**in_params):
    """
    Supported parameters:
    batch_normalization - True or False
    dropout_rate - 0 to 1
    num_units - integer
    learning_rate - float
    activation_function - string
    """
    
    model = Sequential()

    # Set default values
    params = {
        'batch_normalization': False,
        'dropout_rate': 0,
        'num_layers': 6,
        'num_units': 128,
        'learning_rate': 0.001,
        'hidden_activation': 'relu',
        'final_activation': 'softmax'
    }
    
    # Replace defaults with specified parameters
    for param in in_params:
        params[param] = in_params[param]    
    
    # InputLayer causes serialization issues
#     model.add( InputLayer(input_shape = (x_train.shape[1],) ) )
    
#     for i in range(params['num_layers']):
#         addDenseBlock(model, params['num_units'], **params)

    addDenseBlock(model, 128, **params)
    addDenseBlock(model, 128, **params)
    addDenseBlock(model, 256, **params)
    addDenseBlock(model, 256, **params)
    addDenseBlock(model, 128, **params)
    addDenseBlock(model, 128, **params)

    model.add(Dense(units=y_train.shape[1], activation=params['final_activation']))

    # optim = SGD(lr=0.001, nesterov=True)
    optim = Adam(lr=params['learning_rate'])

    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    
#     print(params)
    
    return model

Ensure a GPU is available

In [26]:
import keras.backend as K

K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

### Callbacks

In [30]:
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint

def timestamp():
    return time.strftime("%m-%d-%Y, %I%M%p")

csv_callback = CSVLogger(NOTEBOOK_PATH + 'Loss Logs/%s (%s)' % (model_id, timestamp()), append=True)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta=0.0001, patience=5)

### Model Saving

In [33]:
savedir = NOTEBOOK_PATH + "Models/%s/" % model_id
if not os.path.exists(savedir):
    os.makedirs(savedir)
    print("Created " + savedir)
else:
    print("Using " + savedir)

Created D:/Delta Stuff/Scifair20/Models/dnn1/


### Training

In [35]:
import itertools

trainEpochs = 40

parameters = {'batch_size': 250, 'num_layers':5, 'num_units':256, 'dropout_rate':0}

In [38]:
%%time
print(model_id)
model = createModel(**parameters)
model.summary()
model_checkpoint = ModelCheckpoint(NOTEBOOK_PATH + "Models/%s/dnn({epoch}).h5" % (model_id), monitor='val_loss')
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=parameters['batch_size'], epochs=trainEpochs, verbose=2, callbacks=[csv_callback, model_checkpoint])

dnn1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               8832      
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_17 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_18 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_19 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_20 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_21 (Dense)             (None, 10)                1290      
Total

# F1 Score

In [40]:
from sklearn.metrics import f1_score

In [41]:
model_names = glob.glob(NOTEBOOK_PATH + f"Models/{model_id}/*.h5")
# print(model_names)
print(len(model_names))

40


In [45]:
f1_micro = np.zeros((len(model_names)))
f1_macro = np.zeros((len(model_names)))
for i in tqdm(range(len(model_names))):
    epoch_model = keras.models.load_model(model_names[i])
    pred = epoch_model.predict(x_val).argmax(axis=1)
    pred_f1 = np.zeros((len(x_val), y_val.shape[1]))
    for j in range(len(pred)):
        pred_f1[j,pred[j]] = 1
    f1_micro[i-1] = f1_score(y_val, pred_f1.astype('uint8'), average='micro')
    f1_macro[i-1] = f1_score(y_val, pred_f1.astype('uint8'), average='macro')

  0%|                                                                                           | 0/40 [00:00<?, ?it/s]C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [09:39<00:00, 14.48s/it]


In [47]:
print("F1 Micro")
print(model_names[f1_micro.argmax()])
print(f1_micro.max())

print("F1 Macro")
print(model_names[f1_macro.argmax()])
print(f1_macro.max())

F1 Micro
D:/Delta Stuff/Scifair20/Models/dnn1\dnn(27).h5
0.9982160204892696
F1 Macro
D:/Delta Stuff/Scifair20/Models/dnn1\dnn(17).h5
0.7544766158354702


In [49]:
model = keras.models.load_model("D:/Delta Stuff/Scifair20/Models/dnn1\dnn(27).h5")
pred = model.predict(x_val)

pred_index = list(pred.argmax(1))
y_index = list(np.argmax(y_val.to_numpy(), axis=1))

atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,BENIGN,Bot,Brute Force FTP,Brute Force SSH,DDoS,DoS,Heartbleed,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,,,,
BENIGN,226915,118,0,3,9,37,2,3,0,7,227094
Bot,0,76,0,0,0,0,0,0,0,0,76
Brute Force FTP,1,0,823,5,0,0,0,0,0,0,829
Brute Force SSH,12,0,1,563,0,0,0,0,0,0,576
DDoS,1,0,0,0,12842,0,0,0,0,0,12843
DoS,261,0,1,0,0,25189,0,0,6,2,25459
PortScan,91,0,0,0,0,0,0,0,15904,0,15995
Web Attack,13,0,0,0,0,0,0,0,0,190,203
All,227294,194,825,571,12851,25226,2,3,15910,199,283075


# Log results

In [72]:
model_log = open(NOTEBOOK_PATH + "model_log.txt", "a")

model_log.write("\n" + model_filename)
model_log.write("\n\tF1 Micro: " + str(f1_micro))
model_log.write("\n\tF1 Macro: " + str(f1_macro))

model_log.close()